In [1]:
from scipy.stats import norm
import random
from math import exp, sqrt
import datetime
import numpy as np

def TrinomialTree(isAmerican, isCall, S0, K, r, sigma, T):
    N=95
    dt = T / N  
    u = np.exp(sigma * np.sqrt(2 * dt))
    d = np.exp(-1 * sigma * np.sqrt(2 * dt))


    pu = ((np.exp(r * dt / 2) - np.exp(-1 * sigma * np.sqrt(dt / 2))) / (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-1 * sigma * np.sqrt(dt / 2)))) ** 2
    pd = ((np.exp(sigma * np.sqrt(dt / 2)) - np.exp(r * dt / 2)) / (np.exp(sigma * np.sqrt(dt / 2)) - np.exp(-1 * sigma * np.sqrt(dt / 2)))) ** 2
    pm = 1 - pu - pd

    STs = [np.array([S0])]
    for i in range(N):
        prev_nodes = STs[-1]
        ST = np.concatenate(
            (prev_nodes * u, [prev_nodes[-1] , prev_nodes[-1] * d]))
        STs.append(ST)
        
        
    if isCall:
        payoffs = np.maximum(0, (STs[N] - K))
    else:
        payoffs = np.maximum(0, (K - STs[N]))

    for i in reversed(range(N)):
        payoffs = (payoffs[:-2] * pu + payoffs[1:-1] * pm + payoffs[2:] * pd) * np.exp(-1 * r * dt)
        if isAmerican:
            payoffs = np.maximum(payoffs, STs[i] - K)  # Account for early exercise

    return payoffs[0]


def Vega(isAmerican, isCall, S0, K, r, sigma, T, d=0.01):
    return (TrinomialTree(isAmerican, isCall, S0, K, r, sigma + d, T) -
            TrinomialTree(isAmerican, isCall, S0, K, r, sigma, T)) / d


def find_vol(target_value, isAmerican, isCall, S, K, T, r):
    MAX_ITERATIONS = 100
    PRECISION = 1.0e-5

    sigma = 0.5
    for i in range(0, MAX_ITERATIONS):
        price = TrinomialTree(isAmerican=isAmerican, isCall=isCall, S0=S, K=K, T=T, r=r, sigma=sigma)
        vega = Vega(isAmerican=isAmerican, isCall=isCall, S0=S, K=K, T=T, r=r, sigma=sigma)

        diff = price - target_value

        if abs(diff) < PRECISION:
            return sigma
        sigma = sigma - diff / vega
    return sigma

In [2]:
V_market_and_K = ((11.38, 310), (10.85, 311), (10.21, 312), (9.57, 313), (8.91, 314),(8.28, 315), (7.55, 316), (7.13, 317), (6.53, 318), (5.91, 319), (5.65, 320))
for (a, b) in V_market_and_K:
    vol = find_vol(target_value=a, isAmerican=True, isCall=True, S=311.97, K=b, r=0.0158 - 0.0182, T=0.3695)
    print("Strike: {0}, implied vol: {1:.4f}".format(b, vol))

Strike: 310, implied vol: 0.1390
Strike: 311, implied vol: 0.1387
Strike: 312, implied vol: 0.1369
Strike: 313, implied vol: 0.1345
Strike: 314, implied vol: 0.1317
Strike: 315, implied vol: 0.1294
Strike: 316, implied vol: 0.1253
Strike: 317, implied vol: 0.1249
Strike: 318, implied vol: 0.1222
Strike: 319, implied vol: 0.1189
Strike: 320, implied vol: 0.1200
